In [1]:
import requests
import pandas as pd
import numpy as np
import time
import sklearn
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier 

In [2]:
url = 'https://www.amazon.com/Ray-Ban-Aviator-Sunglasses-Polarized-Crystal/dp/B002ZLYRNI?ref_=Oct_DLandingS_D_a093b838_60&smid=ATVPDKIKX0DER&th=1'
browser = webdriver.Edge(executable_path='./msedgedriver.exe') 
browser.maximize_window()
browser.get(url)

review_page = browser.find_element_by_link_text('See all reviews').click();

In [3]:
reviewer_types = browser.find_elements_by_css_selector('#reviewer-type-dropdown option')
for reviewer_type in reviewer_types:
    if "Verified purchase only" in reviewer_type.text:
        reviewer_type.click()
        break

In [4]:
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All positive" in comment_type.text:
        comment_type.click()
        break

In [5]:
phrases = []
status = []

In [6]:
for i in range(0,4):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('span', {'data-hook': 'review-body'})
        phrases.append(content.text.strip())
        status.append(1)
    browser.find_element_by_class_name('a-last').click()
   

In [7]:
comment_types = browser.find_elements_by_css_selector('#star-count-dropdown option')
for comment_type in comment_types:
    if "All critical" in comment_type.text:
        comment_type.click()
        break

In [8]:
for i in range(0,4):
    time.sleep(3)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    reviews = tree.find_all('div',{'data-hook':'review'})
    for review in reviews:
        content = review.find('span', {'data-hook': 'review-body'})
        phrases.append(content.text.strip())
        status.append(0)
    browser.find_element_by_class_name('a-last').click()
    

In [9]:
Data = pd.DataFrame(
    {'Para': phrases,
     'Stt': status
    })
#Chưa lowercases

In [10]:
Data.to_csv('data.csv',index = False,encoding='utf-16',sep = '\t')

In [29]:
data = pd.read_csv('data.csv',encoding='utf-16',sep = '\t')
phrases = data.Para
status = data.Stt

In [30]:
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords


In [31]:
def stemSentence(sentence):
    sentence = sentence.lower()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        if (word not in stop_words) and (word not in punc):
            stem_sentence.append(englishStemmer.stem(word))
            stem_sentence.append(" ")
    return "".join(stem_sentence)

In [32]:
optimize = []
englishStemmer=SnowballStemmer("english")
punc = "!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
stop_words = set(stopwords.words('english')) 

for phrase in phrases:
    optimize.append(stemSentence(phrase))

In [33]:
vect = CountVectorizer()
vect.fit(optimize)

CountVectorizer()

In [41]:
bag_of_word = vect.transform(optimize)
bag_of_word = bag_of_word.toarray()


In [42]:
train_X, test_X, train_y, test_y = train_test_split(bag_of_word, status, test_size=0.2, 
                                                    random_state=0, stratify=status)

In [36]:
train_y.mean()

0.5

In [37]:
neural_net_model = MLPClassifier(hidden_layer_sizes=(3,), activation='tanh',
                                 solver='lbfgs', max_iter=1000, random_state=0)

In [43]:
neural_net_model.fit(train_X, train_y)

MLPClassifier(activation='tanh', hidden_layer_sizes=(3,), max_iter=1000,
              random_state=0, solver='lbfgs')

In [44]:
# Độ lỗi trên tập huấn luyện
1 - neural_net_model.score(train_X, train_y)

0.0

In [45]:
1 - neural_net_model.score(test_X, test_y)

0.3125

In [26]:
# Tạo mô hình Neural Net với: 
# * 1 tầng ẩn gồm 100 nơ-ron, hàm kích hoạt: tanh
# * Thuật toán cực tiểu hóa: LBFGS, số vòng lặp tối đa: 1000
# * random_state: 0
neural_net_model2 = MLPClassifier(hidden_layer_sizes=(10,), activation='tanh',
                                 solver='lbfgs', max_iter=1000, random_state=0)

In [27]:
neural_net_model2.fit(train_X, train_y)

MLPClassifier(activation='tanh', hidden_layer_sizes=(10,), max_iter=1000,
              random_state=0, solver='lbfgs')

In [28]:
1 - neural_net_model.score(test_X, test_y)

0.3125